In [ ]:
#Bước 1: Import thư viện
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

 19127135_PhamBaoHan_239_Room33.m4a   IMG_0601.mov
 19127135_PhamBaoHan_239_Room33.mp4   IMG_0625.mov
 19127135_PhamBaoHan_434_Room7.m4a    IMG_0645.mov
 19127135_PhamBaoHan_434_Room7.mp4    IMG_0651.jpg
 cat.jpg			      LaMa.gdoc
'Colab Notebooks'		      plant-pathology-2020-fgvc7
 csào.pdf			     'ROOM20_19127135_Phạm Bảo Hân.mp4'
 csáo.pdf			     'RoomX_19127135_Phạm Bảo Hân.mp4'
 IMG_0585.mov			      xyz.jpg


In [ ]:
PATH = '/content/drive/MyDrive/plant-pathology-2020-fgvc7/'
labels = ['healthy', 'multiple_diseases', 'rust', 'scab']

In [ ]:
# Training set and test set
df = pd.read_csv(PATH + 'train.csv')
test_df = pd.read_csv(PATH + 'test.csv')

In [ ]:
df['label'] = df.apply(lambda x: df.columns[x['healthy':'scab'].astype('float').argmax() + 1], axis = 1)

In [ ]:
df['file_name'] = df['image_id'].astype(str)+'.jpg'
test_df['file_name'] = test_df['image_id'].astype(str)+'.jpg'

In [ ]:
df.head()

,image_id,healthy,multiple_diseases,rust,scab,label,file_name
0,Train_0,0,0,0,1,scab,Train_0.jpg
1,Train_1,0,1,0,0,multiple_diseases,Train_1.jpg
2,Train_2,1,0,0,0,healthy,Train_2.jpg
3,Train_3,0,0,1,0,rust,Train_3.jpg
4,Train_4,1,0,0,0,healthy,Train_4.jpg


In [ ]:
#Bước 2: Xác định các thuộc tính hình ảnh - Kích thước, chiều
Image_Channels=3
Image_Width=224
Image_Height=224
Image_Size=(Image_Width,Image_Height)

In [ ]:
#Bước 4: Định nghĩa callbacks và learning rate
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy', 
    mode = 'auto',
    verbose = 2, 
    patience = 10
    )
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = "checkpoint.h5",
    monitor = 'val_accuracy', 
    save_best_only = True, 
    mode = 'auto'
    )

callbacks = [early_stopping, model_checkpoint]

In [ ]:
# plt.figure(figsize=(30, 8))
# for i in range(10):
#   ax = plt.subplot(2, 5, i + 1)
#   file_path = 'images/' + train_x[i] + '.jpg'
#   image = plt.imread(PATH + file_path)
#   plt.imshow(image)
#   plt.axis("off")

In [ ]:
# Bước 5: Quản lý dữ liệu - chia tập validate và train
train_df,validate_df = train_test_split(df,test_size=0.25, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

batch_size=32 

In [ ]:
#Bước 6: Generate tập dữ liệu và điều chỉnh
train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   rotation_range=40,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.1,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_link = PATH + '/images/'

train_generator = train_datagen.flow_from_dataframe(train_df,train_link,x_col='file_name',y_col='label',class_mode='categorical', batch_size=batch_size, target_size=Image_Size, shuffle=True)
validation_generator = val_datagen.flow_from_dataframe(validate_df,train_link,x_col='file_name',y_col='label',class_mode='categorical', batch_size=batch_size, target_size=Image_Size, Shuffle=False)

Found 1365 validated image filenames belonging to 4 classes.
Found 456 validated image filenames belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

In [ ]:
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(Image_Height,Image_Width,3))
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# x = Dense(1024)(x)
# x = LeakyReLU(alpha=0.05)(x)
# x = Dense(512)(x)
# x = LeakyReLU(alpha=0.05)(x)
# x = Dropout(0.2)(x)
# x = Dense(256, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)   


87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
#Bước 8: compile và đánh giá - KHÔNG CHẠY KHI DÙNG MODEL LOAD LẠI
opt=tf.keras.optimizers.SGD(learning_rate=0.0035,momentum=0.9) 

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model_VGG = model.fit(train_generator, 
                              steps_per_epoch=len(train_generator), 
                              validation_data=validation_generator, 
                              validation_steps=len(validation_generator), 
                              epochs=20, 
                              verbose=2,
                              callbacks=callbacks)

Epoch 1/20
43/43 - 870s - loss: 1.6656 - accuracy: 0.6110 - val_loss: 5653.5132 - val_accuracy: 0.3553 - 870s/epoch - 20s/step
Epoch 2/20
43/43 - 681s - loss: 1.0978 - accuracy: 0.8315 - val_loss: 413290.1250 - val_accuracy: 0.2763 - 681s/epoch - 16s/step
Epoch 3/20
43/43 - 685s - loss: 0.6864 - accuracy: 0.8689 - val_loss: 89.9897 - val_accuracy: 0.6469 - 685s/epoch - 16s/step


In [ ]:
print(model_VGG.history.keys())

# Visualize accuracy
plt.plot(model_VGG.history['accuracy'])
plt.plot(model_VGG.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# Visualize loss
plt.plot(model_VGG.history['loss'])
plt.plot(model_VGG.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
submit_datagen = ImageDataGenerator(rescale = 1. / 255)

submit_generator = submit_datagen.flow_from_dataframe(
            dataframe = test_df,
            directory = train_link,
            x_col = 'file_name',
            y_col =  None,    
            target_size = Image_Size,
            batch_size = batch_size,
            class_mode = None,
            classes = ['healthy', 'multiple_diseases', 'rust', 'scab']
)            


In [ ]:
y_pred = model.predict(submit_generator, steps = len(test_df))

In [ ]:
submit = pd.concat([test_df, pd.DataFrame(y_pred, columns=labels) ],  axis = 1)

In [ ]:
submit = submit[['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab' ]]

In [ ]:
submit.head()

In [ ]:
submit.to_csv('Inception-v3.csv', index = False)